In [6]:
import os
import numpy as np
import cv2
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D,BatchNormalization, Activation,MaxPool2D, Conv2DTranspose,Concatenate, Input
from tensorflow.keras.models import load_model


In [7]:
os.environ['PYTHONHASHSEED'] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

In [8]:
batch_size = 16
lr = 1e-4
epochs = 100
height = 224
width = 224

In [9]:
dataset_path = os.path.join("/content/drive/MyDrive/ColabNotebooks/Deeplearning/doan", "dataset")

files_dir = os.path.join("/content/drive/MyDrive/ColabNotebooks/Deeplearning/doan", "weight")
model_file = os.path.join(files_dir, "unet.h5")
log_file = os.path.join(files_dir, "log.csv")

In [10]:
def cerate_dir(path):
  if not os.path.exists(path):
    os.makedirs(path)

In [11]:
cerate_dir(files_dir)

# Build Unet


In [12]:
def conv_block(inputs, num_filters):
  x = Conv2D(num_filters, 3, padding='same')(inputs)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  x = Conv2D(num_filters, 3, padding='same')(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  return x

In [13]:
def encoder_block(inputs, num_filters):
  x = conv_block(inputs, num_filters)
  p = MaxPool2D((2, 2))(x)
  return x, p

In [14]:
def decoder_block(inputs, skip, num_filters):
  x = Conv2DTranspose(num_filters, (2,2), strides=2, padding='same')(inputs)
  x = Concatenate()([x, skip])
  x = conv_block(x, num_filters)
  return x

In [15]:
def build_unet(input_shape):
    inputs = Input(shape=input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding='same', activation='sigmoid')(d4)

    model = Model(inputs, outputs, name="UNET")
    return model


In [16]:
def load_data(path):
  train_x = sorted(glob(os.path.join(path, 'train', 'Images', '*')))
  train_y = sorted(glob(os.path.join(path, 'train', 'Mask', '*')))

  valid_x = sorted(glob(os.path.join(path, 'valid', 'Images', '*')))
  valid_y = sorted(glob(os.path.join(path, 'valid', 'Mask', '*')))

  return (train_x, train_y), (valid_x, valid_y)

In [17]:
def load_data_test(path):
  test_x = sorted(glob(os.path.join(path, 'test', 'Images', '*')))
  test_y = sorted(glob(os.path.join(path, 'test', 'Mask', '*')))

  return (test_x, test_y)

In [18]:
def read_image(path):
  path = path.decode()
  x = cv2.imread(path, cv2.IMREAD_COLOR)
  x = x/255.0
  x = cv2.resize(x, (224, 224))
  return x

In [19]:
def read_mask(path):
  path = path.decode()
  x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
  x = x/255.0
  x = np.expand_dims(x, axis=-1)
  return x

In [20]:
def tf_parse(x, y):
  def _parse(x, y):
    x = read_image(x)
    y = read_mask(y)
    return x, y

  x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
  x.set_shape([height, width, 3])
  y.set_shape([height, width, 1])
  return x, y


In [21]:
def tf_dataset(x, y, batch=16):
  dataset = tf.data.Dataset.from_tensor_slices((x, y))
  dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
  dataset = dataset.batch(batch)
  dataset = dataset.prefetch(tf.data.AUTOTUNE)
  return dataset

In [22]:
(train_x, train_y), (valid_x, valid_y) = load_data(dataset_path)
print(f'Train: {len(train_x)} - {len(train_y)}')
print(f'Valid: {len(valid_x)} - {len(valid_y)}')

Train: 1687 - 1687
Valid: 493 - 493


In [23]:
(test_x, test_y) = load_data_test(dataset_path)
print(f'Train: {len(test_x)} - {len(test_y)}')

Train: 441 - 441


In [24]:
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)
test_dataset = tf_dataset(test_x, test_y, batch=batch_size)

In [25]:
input_shape = (224, 224, 3)
print(input_shape)
model = build_unet(input_shape)

(224, 224, 3)


In [26]:
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 224, 224, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 224, 224, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 224, 224, 64)         0         ['batch_normalization[0][0]

In [27]:
opt = tf.keras.optimizers.Adam(lr)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=['accuracy'])

In [28]:
callbacks = [
    ModelCheckpoint(model_file, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=4),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
]

In [29]:
# H = model.fit(
#     train_dataset,
#     validation_data=valid_dataset,
#     epochs=30,
#     callbacks=callbacks
# )

In [30]:
print(H.history.keys())
plt.figure()
plt.plot(H.history['accuracy'], label="train_acc")
plt.plot(H.history['val_accuracy'], label="val_acc")
plt.plot(H.history['loss'], label = "train_loss")
plt.plot(H.history['val_loss'], label = "val_loss")
plt.title('Training Loss and Accuracy')
plt.ylabel('Loss/Accuracy')
plt.xlabel('Epoch #')
plt.legend()
plt.show()

NameError: name 'H' is not defined

In [ ]:
test_model = load_model("/content/drive/MyDrive/ColabNotebooks/Deeplearning/doan/weight/unet.h5")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

 4/28 [===>..........................] - ETA: 10:21 - loss: 0.0548 - accuracy: 0.9681

In [32]:
test_model = load_model("/content/drive/MyDrive/ColabNotebooks/Deeplearning/doan/weight/unet.h5")


np.random.seed(42)
input_data = np.random.random((1, 512, 512, 3))


output_data = test_model.predict(test_dataset)

print(output_data)

28/28 [==============================] - 725s 26s/step
[[[[0.02073631]
   [0.00343625]
   [0.00601986]
   ...
   [0.00287972]
   [0.00590828]
   [0.03106096]]

  [[0.00326668]
   [0.00083987]
   [0.00188469]
   ...
   [0.00222712]
   [0.004527  ]
   [0.00630655]]

  [[0.00528948]
   [0.00202396]
   [0.00117482]
   ...
   [0.00240906]
   [0.00205616]
   [0.00882585]]

  ...

  [[0.00471038]
   [0.00292212]
   [0.00335729]
   ...
   [0.00101828]
   [0.00177024]
   [0.00712718]]

  [[0.00725625]
   [0.00297162]
   [0.00175604]
   ...
   [0.00183176]
   [0.00384212]
   [0.01595524]]

  [[0.01794057]
   [0.01672317]
   [0.00984155]
   ...
   [0.00424229]
   [0.008834  ]
   [0.04257388]]]


 [[[0.02079521]
   [0.00344779]
   [0.00604922]
   ...
   [0.00282212]
   [0.00581372]
   [0.03045827]]

  [[0.00330416]
   [0.00084256]
   [0.00188907]
   ...
   [0.00214019]
   [0.00446396]
   [0.0061947 ]]

  [[0.00535728]
   [0.00203313]
   [0.00116904]
   ...
   [0.00237708]
   [0.00203797]
   [0.008